In [29]:
import pandas as pd
import numpy as np
from estimator_model.estimation_learner import meta_learner
from estimator_model import ml_model

It's still not clear why XLearner and TLearner are not effective.

In [30]:
n_users = 10000
def gen_data(n_users, binary_treatment=True, with_income=False):
    
    if with_income:
        income = np.random.normal(500, scale=15, size=n_users)
        gender = np.random.randint(0, 2, size=n_users)
        coupon = gender * 20 + 110 + income / 50 + np.random.normal(scale=5, size=n_users)
        if binary_treatment:
            coupon = (coupon > 120).astype(int)
        amount = coupon * 150 + gender * 100 + 150 + income / 5 + np.random.normal(size=n_users)
        time_spent = coupon * 10 + amount / 10

        df = pd.DataFrame({
            'gender': gender,
            'coupon': coupon,
            'amount': amount,
            'income': income,
            'time_spent': time_spent,
        })

        return df
    else:
        gender = np.random.randint(0, 2, size=n_users)
        coupon = gender * 20 + 150 + np.random.normal(scale=5, size=n_users)
        if binary_treatment:
            coupon = (coupon > 150).astype(int)
        amount = coupon * 30 + gender * 100 + 150 + np.random.normal(size=n_users)
        time_spent = coupon * 100 + amount / 10

        df = pd.DataFrame({
            'gender': gender,
            'coupon': coupon,
            'amount': amount,
            'time_spent': time_spent,
        })
        
        return df

df = gen_data(n_users, with_income=True)
df['treatment'] = df['coupon'] > 0
df

,gender,coupon,amount,income,time_spent,treatment
0,1,1,497.061194,478.662101,59.706119,True
1,0,1,401.661125,504.007319,50.166112,True
2,1,1,504.975703,526.540901,60.497570,True
3,0,0,245.920657,485.997889,24.592066,False
4,0,1,402.105066,511.450147,50.210507,True
...,...,...,...,...,...,...
9995,1,1,504.298182,533.415845,60.429818,True
9996,0,0,249.256014,494.930205,24.925601,False
9997,0,1,401.828561,507.329570,50.182856,True
9998,1,1,499.112329,503.298929,59.911233,True


In [19]:
new_df = df.drop('treatment', axis=1)

In [37]:
a = np.array([i for i in df['income']])

In [38]:
a

array([478.66210148, 504.00731885, 526.54090084, ..., 507.32957032,
       503.29892927, 499.13256306])

In [41]:
(df['income'] / df['income']).mean()

1.0

In [20]:
new_df[new_df['time_spent'] > 26]['coupon'].value_counts()

1    7505
0       2
Name: coupon, dtype: int64

## SLearner

In [21]:
ml = ml_model.LR()
l = meta_learner.SLearner(ml_model=ml)

In [22]:
l.prepare(data=new_df, outcome='amount', treatment='coupon', adjustment={'gender', 'income'})

array([150.02196995, 150.02196995, 150.02196995, ..., 150.02196995,
       150.02196995, 150.02196995])

In [23]:
l.estimate_ate(data=new_df, 
               outcome='amount', 
               treatment='coupon', 
               adjustment=['gender', 'income'])

150.02196994838965

In [24]:
l.estimate_cate(data=new_df, 
                outcome='amount', 
                treatment='coupon', 
                adjustment=['gender', 'income'],
                condition_set={'time_spent'},
                condition=(new_df['time_spent'] >= 25.06))

149.71783769915066

## TLearner

In [25]:
t = meta_learner.TLearner(ml_model=ml)
t.estimate_ate(data=new_df, 
               outcome='amount', 
               treatment='coupon', 
               adjustment=['gender', 'income'])

199.63088066696852

In [26]:
t.estimate_cate(data=new_df, 
                outcome='amount', 
                treatment='coupon', 
                adjustment=['gender', 'income'],
                condition_set={'time_spent'},
                condition=(new_df['time_spent'] >= 20))

199.63088066696852

## XLearner

In [27]:
x = meta_learner.XLearner(ml_model=ml)
x.estimate_ate(data=new_df, 
               outcome='amount', 
               treatment='coupon', 
               adjustment=['gender', 'income'])

24.804423189303208

$\bot$

In [28]:
a = '-'
for i in range(9):
    a += '-'
a

'----------'